In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras

(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_full = x_train_full / 255.0
x_test = x_test / 255.0
x_valid, x_train = x_train_full[:5000], x_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

pixel_means = x_train.mean(axis=0, keepdims=True)
pixel_stds = x_train.std(axis=0, keepdims=True)
x_train_scaled = (x_train - pixel_means) / pixel_stds
x_valid_scaled = (x_valid - pixel_means) / pixel_stds
x_test_scaled = (x_test - pixel_means) / pixel_stds

$\ell_1$  and  $\ell_2$ Regularization

In [2]:
# l1 regularization
layer = keras.layers.Dense(100, activation='elu', 
                          kernel_initializer='he_normal', 
                          kernel_regularizer=keras.regularizers.l1(0.1))

# l2 regularization
layer = keras.layers.Dense(100, activation='elu', 
                          kernel_initializer='he_normal', 
                          kernel_regularizer=keras.regularizers.l2(0.01))

# l1 and l2 rgularization
layer = keras.layers.Dense(100, activation='elu', 
                          kernel_initializer='he_normal', 
                          kernel_regularizer=keras.regularizers.l1_l2(l1=0.1, l2=0.01))

In [3]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='elu', 
                       kernel_initializer='he_normal', 
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation='elu', 
                       kernel_initializer='he_normal', 
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation='softmax', 
                       kernel_regularizer=keras.regularizers.l2(0.01))    
])

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', metrics=['acc'])
n_epochs = 2
history = model.fit(x_train_scaled, y_train, epochs=n_epochs, 
                    validation_data=(x_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 4s 2ms/step - loss: 1.6272 - acc: 0.8129 - val_loss: 0.7527 - val_acc: 0.8088
Epoch 2/2
1719/1719 [==============================] - 4s 2ms/step - loss: 0.7193 - acc: 0.8255 - val_loss: 0.7031 - val_acc: 0.8378


In [4]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense, activation='elu', 
                           kernel_initializer='he_normal', 
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', metrics=['acc'])
n_epochs = 2
history = model.fit(x_train_scaled, y_train, epochs=n_epochs, 
                    validation_data=(x_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 4s 2ms/step - loss: 1.6641 - acc: 0.8137 - val_loss: 0.7324 - val_acc: 0.8168
Epoch 2/2
1719/1719 [==============================] - 4s 2ms/step - loss: 0.7211 - acc: 0.8254 - val_loss: 0.6875 - val_acc: 0.8366


Dropout

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation='elu', 
                       kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.2), 
    keras.layers.Dense(100, activation='elu', 
                       kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation='softmax')    
])

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', metrics=['acc'])
n_epochs = 2
history = model.fit(x_train_scaled, y_train, epochs=n_epochs, 
                    validation_data=(x_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5795 - acc: 0.7996 - val_loss: 0.3749 - val_acc: 0.8614
Epoch 2/2
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4219 - acc: 0.8444 - val_loss: 0.3643 - val_acc: 0.8612


Alpha Dropout

In [6]:
tf.random.set_seed(42)
np.random.seed(42)

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation='selu', 
                       kernel_initializer='lecun_normal'),
    keras.layers.AlphaDropout(rate=0.2), 
    keras.layers.Dense(100, activation='selu', 
                       kernel_initializer='lecun_normal'),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation='softmax')    
])

optimizer = keras.optimizers.SGD(learning_rate=0.01, 
                                 momentum=0.9, 
                                 nesterov=True)
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=optimizer, metrics=['acc'])
n_epochs = 20
history = model.fit(x_train_scaled, y_train, epochs=n_epochs, 
                    validation_data=(x_valid_scaled, y_valid))

Epoch 1/20
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6640 - acc: 0.7593 - val_loss: 0.5783 - val_acc: 0.8446
Epoch 2/20
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5566 - acc: 0.7945 - val_loss: 0.5181 - val_acc: 0.8514
Epoch 3/20
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5205 - acc: 0.8084 - val_loss: 0.4899 - val_acc: 0.8614
Epoch 4/20
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5084 - acc: 0.8117 - val_loss: 0.4846 - val_acc: 0.8604
Epoch 5/20
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4949 - acc: 0.8179 - val_loss: 0.4234 - val_acc: 0.8682
Epoch 6/20
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4831 - acc: 0.8198 - val_loss: 0.4596 - val_acc: 0.8622
Epoch 7/20
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4726 - acc: 0.8242 - val_loss: 0.4686 - val_acc: 0.8610
Epoch 8/20
1719/1719 [==============================] - 3s 2ms

In [8]:
model.evaluate(x_test_scaled, y_test)

313/313 [==============================] - 0s 686us/step - loss: 0.4873 - acc: 0.8566


[0.48730704188346863, 0.8565999865531921]

In [9]:
model.evaluate(x_train_scaled, y_train)

1719/1719 [==============================] - 1s 655us/step - loss: 0.3619 - acc: 0.8801


[0.36193376779556274, 0.8800545334815979]

In [10]:
history = model.fit(x_train_scaled, y_train)

1719/1719 [==============================] - 3s 2ms/step - loss: 0.4226 - acc: 0.8427
